# Data analysis

We would like to find statistically significant changes in the data after debris reentry. To do this we will first do the following analysis:

1. Statistical Differences (before & after reentry)
    - Mean
    - Median
    - Standard Deviation
2. Anomaly Detection
3. Range of change
4. Cumulative Unsigned Rate of Change

Finally, we will train an AI model on the data or some selected metrics and we will see how well it performs in predicting reentry events.

In [4]:
import pandas as pd

# Load the CSV file
file_path = "outputs/output.csv"
df = pd.read_csv(file_path)

# Display headers and first few rows
df_head = df.head()

# Display last few rows to get an overall sense of the data
df_tail = df.tail()

df_head, df_tail


(                  date      object  airtempsCurrentTemp  \
 0  2020-03-10T02:30:00  NORAD 6792                 31.7   
 1  2020-03-10T05:00:00  NORAD 6792                 29.2   
 2  2020-03-10T05:30:00  NORAD 6792                 28.4   
 3  2020-03-10T08:30:00  NORAD 6792                 25.4   
 4  2020-03-10T09:00:00  NORAD 6792                 23.4   
 
    dissolvedoxygensDissolvedOxygenViaKitMgl  \
 0                                       NaN   
 1                                       NaN   
 2                                       NaN   
 3                                       NaN   
 4                                       NaN   
 
    dissolvedoxygensDissolvedOxygenViaProbeMgl  \
 0                                         NaN   
 1                                         NaN   
 2                                         NaN   
 3                                         NaN   
 4                                         NaN   
 
    dissolvedoxygensSalinityViaDokitPpt    dis

In [5]:
# Extract the object numbers from the re-entry log
reentry_objects = set()  # Using a set for quick lookup

with open("outputs/near-reentry-log.txt", "r") as file:
    for line in file:
        parts = line.split()
        if len(parts) > 1 and parts[1].isdigit():
            reentry_objects.add(parts[1])  # Extract the object number

# Extract object numbers from the dataset
df["object_id"] = df["object"].str.extract(r'(\d+)')  # Extract numeric part of "object"
df["object_id"] = df["object_id"].astype(str)

# Find matching objects in the dataset
matching_objects = df[df["object_id"].isin(reentry_objects)]["object_id"].unique()

# Count how many re-entry objects have matching data
num_matches = len(matching_objects)
num_total = len(reentry_objects)

num_matches, num_total, matching_objects

(108,
 241,
 array(['6792', '9911', '25274', '25469', '26476', '26640', '26692',
        '27613', '28130', '28418', '28641', '28921', '29247', '29386',
        '29482', '29512', '29664', '32751', '32765', '34603', '36749',
        '38037', '38047', '38048', '38872', '38994', '39069', '39083',
        '39259', '39482', '39559', '39566', '39572', '39649', '40313',
        '40422', '40729', '40913', '40963', '41107', '41574', '41575',
        '41576', '41640', '41777', '41839', '41865', '42704', '42903',
        '42972', '43101', '43624', '43673', '43684', '44251', '44385',
        '44820', '44924', '44939', '45076', '45091', '45103', '45203',
        '45392', '45612', '45704', '45736', '46365', '46368', '46536',
        '46673', '46769', '47168', '47396', '47569', '47570', '47605',
        '47741', '47801', '47999', '48409', '48579', '48665', '48809',
        '49223', '49339', '49386', '49524', '49923', '50164', '51967',
        '52279', '52512', '52691', '53270', '53759', '53808', '5560

Now that we have the data for reentry time matched to our extracted GLOBE measurements, we can proceed with further analysis.

In [6]:
from datetime import datetime, timedelta

# Convert date columns to datetime
df["date"] = pd.to_datetime(df["date"])

reentry_times = {}

with open("outputs/near-reentry-log.txt", "r") as file:
    for line in file:
        parts = line.strip().rstrip(".").split()  # Remove trailing period and split
        if len(parts) > 1 and parts[1].isdigit():
            object_id = parts[1]
            timestamp_str = f"{parts[-2]} {parts[-1]}"  # Extract date and time
            try:
                reentry_times[object_id] = datetime.strptime(timestamp_str, "%Y-%m-%d %H:%M:%S")
            except ValueError as e:
                print(f"Error parsing timestamp for object {object_id}: {timestamp_str} -> {e}")

# Check if parsing succeeded
len(reentry_times)

241

In [32]:
# Define time windows before and after reentry (24 and 48 hours)
time_windows = [72]

# Create dictionaries to store time series data
time_series_data = {win: {"before": [], "after": [], "distance": []} for win in time_windows}

# Extract data within the time windows
for obj_id in matching_objects:
    if obj_id in reentry_times:
        reentry_time = reentry_times[obj_id]

        for win in time_windows:
            time_before = reentry_time - timedelta(hours=win)
            time_after = reentry_time + timedelta(hours=win)

            # Select data within the time window
            before_reentry = df[(df["object_id"] == obj_id) & (df["date"] >= time_before) & (df["date"] < reentry_time)]
            after_reentry = df[(df["object_id"] == obj_id) & (df["date"] > reentry_time) & (df["date"] <= time_after)]

            time_series_data[win]["before"].append(before_reentry)
            time_series_data[win]["after"].append(after_reentry)
            time_series_data[win]["distance"].append(df[["distance", "object_id"]][df["object_id"] == obj_id])


# Concatenate results into DataFrames
for win in time_windows:
    time_series_data[win]["before"] = pd.concat(time_series_data[win]["before"])
    time_series_data[win]["after"] = pd.concat(time_series_data[win]["after"])
    time_series_data[win]["distance"] = pd.concat(time_series_data[win]["distance"])

# Display a summary of extracted data
{win: {"before_count": len(time_series_data[win]["before"]), "after_count": len(time_series_data[win]["after"]), "distance_count": len(time_series_data[win]["distance"])} for win in time_windows}


{72: {'before_count': 41480, 'after_count': 41061, 'distance_count': 395259}}

In [ ]:
import random

# Randomly select 5 objects from the matching ones
random_objects = random.sample(list(matching_objects), 5)

random_objects


['29247', '29512', '55606', '39083', '48409']

In [38]:
import numpy as np

# Function to compute baseline statistics for numeric columns
def compute_stats(df):
    # Select only numeric columns
    df_numeric = df.select_dtypes(include=[np.number])
    df_numeric.dropna(inplace=True, axis=1, how="all")
    
    stats = {
        "mean": df_numeric.mean(),
        "median": df_numeric.median(),
        "std_dev": df_numeric.std(),
        "IQR": df_numeric.quantile(0.75) - df_numeric.quantile(0.25),
        "range": df_numeric.max() - df_numeric.min()
    }
    return stats

# Prepare a dictionary to store the results by object
baseline_stats_by_object = {}

for win in time_windows:
    for obj_id in matching_objects:
        # Get the data for the current object
        before_data = time_series_data[win]["before"][time_series_data[win]["before"]["object_id"] == obj_id]
        after_data = time_series_data[win]["after"][time_series_data[win]["after"]["object_id"] == obj_id]
        distance = time_series_data[win]["distance"][time_series_data[win]["distance"]["object_id"] == obj_id]["distance"].mean()
        
        if obj_id not in baseline_stats_by_object:
            baseline_stats_by_object[obj_id] = {}
        
        baseline_stats_by_object[obj_id][win] = {
            "before": compute_stats(before_data),
            "after": compute_stats(after_data),
            "distance": distance
        }

baseline_stats_by_object


{'6792': {72: {'before': {'mean': airtempsCurrentTemp                   30.063121
    distance                             998.315049
    humiditiesDewpoint                    21.869286
    humiditiesRelativeHumidityPercent     66.178571
    dtype: float64,
    'median': airtempsCurrentTemp                   27.900000
    distance                             998.651955
    humiditiesDewpoint                    23.400000
    humiditiesRelativeHumidityPercent     75.500000
    dtype: float64,
    'std_dev': airtempsCurrentTemp                   4.584655
    distance                              4.000536
    humiditiesDewpoint                    3.004598
    humiditiesRelativeHumidityPercent    23.419929
    dtype: float64,
    'IQR': airtempsCurrentTemp                   8.700
    distance                              0.000
    humiditiesDewpoint                    3.475
    humiditiesRelativeHumidityPercent    44.250
    dtype: float64,
    'range': airtempsCurrentTemp                  

In [39]:
import pandas as pd
import numpy as np
from tabulate import tabulate  # used to output markdown tables

def calculate_stats_series(series, trim_percentile=5):
    """Calculate statistics for a pandas Series after removing outliers based on a given percentile and return a dictionary."""
    
    # Sort the series to find the cut-off points for trimming
    series_sorted = series.sort_values()
    
    # Calculate the trim cut-off points based on the given percentile
    lower_trim = series_sorted.quantile(trim_percentile / 100)
    upper_trim = series_sorted.quantile(1 - trim_percentile / 100)
    
    # Remove outliers based on the trim values (both lower and upper)
    trimmed_series = series_sorted[
        (series_sorted >= lower_trim) & (series_sorted <= upper_trim)
    ]
    
    # Calculate statistics on the trimmed data
    stats = {
        "mean": trimmed_series.mean(),
        "median": trimmed_series.median(),
        "std_dev": trimmed_series.std(),
        "IQR": trimmed_series.quantile(0.75) - trimmed_series.quantile(0.25),
        "range": trimmed_series.max() - trimmed_series.min(),
    }
    
    return stats


# Dictionary to store the comparison tables
comparison_tables = {}

# Iterate over time windows and objects
for win in time_windows:
    for obj_id in matching_objects:
        # Filter before and after data for this object
        before_data = time_series_data[win]["before"][time_series_data[win]["before"]["object_id"] == obj_id]
        after_data  = time_series_data[win]["after"][time_series_data[win]["after"]["object_id"] == obj_id]
        distance = time_series_data[win]["distance"][time_series_data[win]["distance"]["object_id"] == obj_id]["distance"].mean()
        
        # Proceed only if there is data in both periods
        if not before_data.empty and not after_data.empty:
            # Determine which numeric measurements are available
            numeric_cols = before_data.select_dtypes(include=[np.number]).columns
            for column in numeric_cols:
                # Get the measurement series for before and after (dropping NaN)
                before_series = before_data[column].dropna()
                after_series  = after_data[column].dropna()
                
                if not before_series.empty and not after_series.empty:
                    # Compute statistics for each measurement
                    before_stats = calculate_stats_series(before_series)
                    after_stats  = calculate_stats_series(after_series)
                    
                    # Build a DataFrame for this measurement's comparison
                    comp_df = pd.DataFrame({
                        "Before": pd.Series(before_stats),
                        "After": pd.Series(after_stats)
                    })
                    comp_df.index.name = "Measurement"

                    # Add a Δ column
                    comp_df["Δ"] = comp_df["After"] - comp_df["Before"]
                    
                    # Build a table name that includes the object and measurement name
                    table_name = f"Object {obj_id} - {column} - Window {win} hours - Measured {distance}km Away"
                    comparison_tables[table_name] = comp_df

In [40]:
# Output each table in Markdown format
for table_name, df in comparison_tables.items():
    print(f"### Comparison Table: {table_name}")
    print(df.to_markdown(tablefmt="github", floatfmt=".2f"))
    print("\n")


### Comparison Table: Object 6792 - airtempsCurrentTemp - Window 72 hours - Measured 998.3148785401547km Away
| Measurement   |   Before |   After |     Δ |
|---------------|----------|---------|-------|
| mean          |    29.91 |   26.37 | -3.54 |
| median        |    27.90 |   25.50 | -2.40 |
| std_dev       |     4.29 |    3.32 | -0.97 |
| IQR           |     8.15 |    5.15 | -3.00 |
| range         |    12.40 |   12.60 |  0.20 |


### Comparison Table: Object 6792 - distance - Window 72 hours - Measured 998.3148785401547km Away
| Measurement   |   Before |   After |     Δ |
|---------------|----------|---------|-------|
| mean          |   998.65 |  998.65 |  0.00 |
| median        |   998.65 |  998.65 |  0.00 |
| std_dev       |     0.00 |    0.00 | -0.00 |
| IQR           |     0.00 |    0.00 |  0.00 |
| range         |     0.00 |    0.00 |  0.00 |


### Comparison Table: Object 6792 - humiditiesDewpoint - Window 72 hours - Measured 998.3148785401547km Away
| Measurement   |   

In [ ]:
# Initialize a structure to hold the changes
aggregated_changes = {}

for win in time_windows:
    for obj_id in matching_objects:
        before_data = time_series_data[win]["before"][time_series_data[win]["before"]["object_id"] == obj_id]
        after_data = time_series_data[win]["after"][time_series_data[win]["after"]["object_id"] == obj_id]
        distance = time_series_data[win]["distance"][time_series_data[win]["distance"]["object_id"] == obj_id]["distance"].mean()

        if not before_data.empty and not after_data.empty:
            numeric_cols = before_data.select_dtypes(include=[np.number]).columns

            for column in numeric_cols:
                # Drop NaN values explicitly before statistic calculations
                before_series = before_data[column].dropna()
                after_series = after_data[column].dropna()

                # Ensure we have enough data after dropping NaNs for meaningful calculations
                if len(before_series) > 1 and len(after_series) > 1:
                    before_stats = calculate_stats_series(before_series)
                    after_stats = calculate_stats_series(after_series)

                    # Handle potential NaNs in specific statistics
                    if 'std_dev' in before_stats and 'std_dev' in after_stats:
                        if np.isnan(before_stats['std_dev']) or np.isnan(after_stats['std_dev']):
                            continue  # Skip further processing for cases with NaN std_dev
                        
                    stats_difference = {key: after_stats[key] - before_stats[key] for key in before_stats}
                    
                    if column not in aggregated_changes:
                        aggregated_changes[column] = {key: [] for key in stats_difference}
                        
                    for key in stats_difference:
                        aggregated_changes[column][key].append(stats_difference[key])
# Compute aggregate statistics, e.g., mean and standard deviation of changes
for column, changes in aggregated_changes.items():
    for stat, values in changes.items():
        values_np = np.array(values)
        aggregated_changes[column][stat] = {
            "mean_change": np.mean(values_np),
            "stddev_change": np.std(values_np)
        }

# Display the aggregated changes
for column, stats in aggregated_changes.items():
    print(f"### Aggregated Changes for Measurement: {column}")
    for stat, data in stats.items():
        mean_change = data['mean_change']
        stddev_change = data['stddev_change']

        print(f"{stat}: Mean Change = {mean_change:.2f}, Std Dev of Change = {stddev_change:.2f}")
    print("\n")

KeyboardInterrupt: 

In [43]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.colors as mcolors
import matplotlib.cm as cm

# Data storage
hist_data = {}

for win in time_windows:
    for obj_id in matching_objects:
        before_data = time_series_data[win]["before"][time_series_data[win]["before"]["object_id"] == obj_id]
        after_data = time_series_data[win]["after"][time_series_data[win]["after"]["object_id"] == obj_id]
        distance = time_series_data[win]["distance"][time_series_data[win]["distance"]["object_id"] == obj_id]["distance"].mean()
        
        if not before_data.empty and not after_data.empty:
            numeric_cols = before_data.select_dtypes(include=[np.number]).columns

            for column in numeric_cols:
                before_stats = calculate_stats_series(before_data[column].dropna())
                after_stats = calculate_stats_series(after_data[column].dropna())

                stats_difference = {key: after_stats[key] - before_stats[key] for key in before_stats}
                stats_difference["distance"] = distance  # Store distance
                
                if column in hist_data:
                    hist_data[column].append(stats_difference)
                else:
                    hist_data[column] = [stats_difference]

# Function to plot histograms with bin colors based on distance

def plot_histograms(data, value, pdf=None):
    for key, values in data.items():
        means = [entry[value] for entry in values if not np.isnan(entry[value])]
        distances = [entry["distance"] for entry in values if not np.isnan(entry[value])]

        if len(means) < 16:
            continue

        if means:
            fig, ax = plt.subplots(figsize=(8, 6))
            counts, bins, patches = ax.hist(means, bins=10, edgecolor='black')

            norm = mcolors.Normalize(vmin=min(distances), vmax=max(distances))
            colormap = cm.viridis
            
            for bin_patch, dist in zip(patches, distances):
                bin_patch.set_facecolor(colormap(norm(dist)))

            sm = cm.ScalarMappable(cmap=colormap, norm=norm)
            sm.set_array([])
            cbar = fig.colorbar(sm, ax=ax, label="Distance")
            
            ax.set_title(f"{value} Difference of {key}")
            ax.set_xlabel("Delta")
            ax.set_ylabel("Frequency")
            
            if pdf:
                pdf.savefig(fig)
                plt.close(fig)
            else:
                plt.show()

with PdfPages("outputs/histograms.pdf") as pdf:
    plot_histograms(hist_data, "mean", pdf)
    plot_histograms(hist_data, "std_dev", pdf)
    plot_histograms(hist_data, "range", pdf)
